In [1]:
import time
import requests
import pandas as pd

env = 'dev'

filter_label = catalog.load('params:openaire_fetch_options.filter_label')
filter_param = catalog.load('params:openaire_fetch_options.filter_param')
filter_value = catalog.load('params:openaire_fetch_options.filter_value')
access_token = catalog.load('params:openaire_fetch_options.access_token')
refresh_token = catalog.load('params:openaire_fetch_options.refresh_token')

                    INFO     Loading data from params:openaire_fetch_options.filter_label       ]8;id=754128;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=747790;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.filter_param       ]8;id=293071;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=586539;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.filter_value       ]8;id=886964;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=686076;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.access_token       ]8;id=267110;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=115054;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:openaire_fetch_options.refresh_token      ]8;id=354402;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=26976;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [2]:
def refresh_access_token(refresh_token):
    """Obtiene un nuevo access_token usando el refresh_token."""
    refresh_url = f"https://services.openaire.eu/uoa-user-management/api/users/getAccessToken?refreshToken={refresh_token}"
    response = requests.get(refresh_url)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"Failed to refresh token: {response.status_code}")

In [3]:
def fetch_openaire_researchproduct(filter_label, filter_param, filter_value, access_token, refresh_token, env):
    cursor = '*'
    base_url = 'https://api.openaire.eu/graph/researchProducts'
    iteration_limit = 5
    iteration_count = 0
    page_size = 50         # Ajustar según sea necesario
    max_retries = 5        # Máximo número de reintentos en caso de error 429
    retry_wait = 5         # Tiempo inicial de espera entre reintentos (segundos)
    max_refresh_attempts = 3  # Máximo número de intentos para refrescar el token
    refresh_attempts = 0

    request_headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    query_params = {
        filter_param: filter_value,
        "pageSize": page_size,
        "cursor": cursor
    }

    while True:
        response = requests.get(base_url, headers=request_headers, params=query_params)

        # Si el token es inválido o expiró, intentar renovarlo
        if response.status_code == 403:
            if refresh_attempts >= max_refresh_attempts:
                raise Exception("Máximo de intentos para refrescar el token alcanzado. Abortando.")
            print("Access token expired or invalid. Refreshing token...")
            new_token = refresh_access_token(refresh_token)
            if not new_token:
                raise Exception("No se pudo refrescar el access token.")
            access_token = new_token
            request_headers["Authorization"] = f"Bearer {access_token}"
            refresh_attempts += 1
            continue  # Reintenta la solicitud con el nuevo token

        if response.status_code != 200:
            raise Exception(f"Failed to retrieve data: {response.status_code}")

        # Restablecemos el contador de refrescos al tener una respuesta exitosa
        refresh_attempts = 0

        api_response = response.json()
        print(f"Iteration count: {iteration_count}")
        print(f"GET {response.url}")

        # Crear DataFrame con el primer bloque de resultados
        df = pd.DataFrame.from_dict(api_response["results"])

        # Actualizar cursor
        cursor = api_response["header"].get("nextCursor", None)
        query_params["cursor"] = cursor

        # Bucle para iterar con el cursor
        while cursor:
            if env == "dev" and iteration_count >= iteration_limit:
                break

            iteration_count += 1
            print(f"Iteration count: {iteration_count}")
            print(f"GET {response.url}")
            time.sleep(2)

            # Reintentos en caso de error 429
            retries = 0
            while retries < max_retries:
                response = requests.get(base_url, headers=request_headers, params=query_params)

                if response.status_code == 403:
                    if refresh_attempts >= max_refresh_attempts:
                        raise Exception("Máximo de intentos para refrescar el token alcanzado durante la ejecución. Abortando.")
                    print("Access token expired during execution. Refreshing token...")
                    new_token = refresh_access_token(refresh_token)
                    if not new_token:
                        raise Exception("No se pudo refrescar el access token durante la ejecución.")
                    access_token = new_token
                    request_headers["Authorization"] = f"Bearer {access_token}"
                    refresh_attempts += 1
                    continue  # Reintenta con el nuevo token

                if response.status_code == 429:
                    retries += 1
                    print(f"Rate limit hit. Retry {retries}/{max_retries}. Waiting {retry_wait} seconds...")
                    time.sleep(retry_wait)
                    retry_wait *= 2  # Incremento exponencial del tiempo de espera
                else:
                    break

            if response.status_code != 200:
                print(f"Failed to retrieve data at iteration {iteration_count}: {response.status_code}")
                break

            # Restablecer contador de refrescos tras respuesta exitosa
            refresh_attempts = 0

            api_response = response.json()

            if not api_response.get("results"):
                print("No more results. Stopping iteration.")
                break

            df_tmp = pd.DataFrame.from_dict(api_response["results"])
            df = pd.concat([df, df_tmp], ignore_index=True)

            cursor = api_response["header"].get("nextCursor", None)
            query_params["cursor"] = cursor

        # Suponiendo que se quiera marcar con la variable filter_value; de lo contrario, ajusta según corresponda.
        df['filter_applied'] = filter_label

        return df, df.head(1000)


In [4]:
df, df_dev = fetch_openaire_researchproduct(filter_label, filter_param, filter_value, access_token, refresh_token, env)

Access token expired or invalid. Refreshing token...
Iteration count: 0
GET https://api.openaire.eu/graph/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=%2A
Iteration count: 1
GET https://api.openaire.eu/graph/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=%2A
Iteration count: 2
GET https://api.openaire.eu/graph/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=AoI%2FD2RlZHVwX3dmXzAwMjo6MDAzOTVjZDIxYTA1ZDI0YTBhMjc3ZGRlNzBlM2MyZjQIP4AAAA%3D%3D
Iteration count: 3
GET https://api.openaire.eu/graph/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pageSize=50&cursor=AoI%2FD2RlZHVwX3dmXzAwMjo6MDA3MTZmMjkxNGE0NGMyMzA2YWI4MDMzMjgwZWQ1ODAIP4AAAA%3D%3D
Iteration count: 4
GET https://api.openaire.eu/graph/researchProducts?relOrganizationId=openorgs____%3A%3A40b9f835648a3e0d057d6917dd7e54d5&pa

In [5]:
df

,author,openAccessColor,publiclyFunded,type,language,subjects,mainTitle,description,publicationDate,publisher,...,originalId,indicators,instance,isGreen,isInDiamondJournal,contributor,pid,country,contactPerson,is_unlp
0,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}","[{'subject': {'scheme': 'keyword', 'value': 'g...",Variability in the growth rates of Saanen kids...,[This study was carried out to determine the v...,2023-12-18,"Universidad Nacional Mayor de San Marcos, Facu...",...,"[oai:ojs.csi.unmsm:article/25240, 50|4dc99724c...","{'citationImpact': {'citationCount': 0.0, 'inf...","[{'alternateIdentifier': [{'scheme': 'doi', 'v...",False,False,NaN,NaN,NaN,NaN,True
1,"[{'fullName': 'Evangelista, Carolina', 'name':...",None,False,publication,"{'code': 'Español', 'label': 'Español'}","[{'subject': {'scheme': 'keyword', 'value': 'C...",Estudio de la viabilidad del algoritmo super-t...,[En este trabajo se presenta el diseño de un c...,2013-01-01,NaN,...,[50|RECOLECTA___::24cb4438d1afe299e63cfdea4a31...,"{'citationImpact': {'citationCount': 0.0, 'inf...","[{'pid': [{'scheme': 'handle', 'value': '10261...",True,False,"[European Commission, Ministerio de Economía y...","[{'scheme': 'handle', 'value': '10261/97737'}]",NaN,NaN,True
2,"[{'fullName': 'Mostoghiu, Robert', 'name': 'Ro...",None,False,publication,"{'code': 'eng', 'label': 'English'}","[{'subject': {'scheme': 'keyword', 'value': 'S...",The Three Hundred project: The gas disruption ...,[We analyse the gas content evolution of infal...,2021-03-01,Oxford University Press (OUP),...,[oai:nottingham-repository.worktribe.com:53501...,"{'citationImpact': {'citationCount': 0.0, 'inf...","[{'type': 'Article', 'url': ['https://nottingh...",True,False,NaN,NaN,NaN,NaN,True
3,"[{'fullName': 'Rubio, Damián', 'name': 'Damiá...",None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'subject': {'scheme': 'keyword', 'value': 'G...",Evolución del sistema de gestión de incidentes...,[Esta tesina detalla el desarrollo de un siste...,2023-06-23,NaN,...,"[oai:sedici.unlp.edu.ar:10915/154923, 50|od___...","{'citationImpact': {'citationCount': 0.0, 'inf...","[{'type': 'Doctoral thesis', 'url': ['http://s...",True,False,"[Lanfranco, Einar Felipe, Venosa, Paula]",NaN,"[{'code': 'AR', 'label': 'Argentina', 'provena...",NaN,True
4,"[{'fullName': 'Ojeda Hidalgo, José F.', 'name'...",gold,False,publication,"{'code': 'eng', 'label': 'English'}","[{'subject': {'scheme': 'keyword', 'value': 'A...",Responsabilidad Social Corporativa en Empresas...,[El presente artículo presenta un conjunto de ...,2013-12-01,Universidad Nacional de La Plata,...,[oai:doaj.org/article:a86e0a7dd7ad4000ae40ad07...,"{'citationImpact': {'citationCount': 0.0, 'inf...","[{'type': 'Article', 'url': ['https://doaj.org...",False,False,NaN,NaN,"[{'code': 'AR', 'label': 'Argentina', 'provena...",NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"[{'fullName': 'Lasta, Jorge Augusto', 'name': ...",None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'subject': {'scheme': 'keyword', 'value': 'C...",Respuesta inmunológica de bovinos vacunados si...,[Se estudia la respuesta inmunológica de bovin...,1978-01-01,NaN,...,[50|od______1329::3d938f84a5fcb447e48259a71e15...,"{'citationImpact': {'citationCount': 0.0, 'inf...","[{'type': 'Doctoral thesis', 'url': ['http://s...",True,False,"[Gimeno, Emilio Juan]",NaN,"[{'code': 'AR', 'label': 'Argentina', 'provena...",NaN,True
296,[{'fullName': 'Revista Institucional de la Fac...,None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'subject': {'scheme': 'keyword', 'value': 'C...","Workshop: ""Fluctuaciones macroeconómicas y cri...",[Perry G. Mehrling es doctor en Economía de la...,2014-12-01,NaN,...,[50|od______1329::bd16c62b0c656e595ffdbc725b03...,"{'citationImpact': {'citationCount': 0.0, 'inf...","[{'type': 'Article', 'url': ['http:/